# CAS Data Engineering FHNW
## Data Wrangling Module - Author F.Burnelli


### Exercise 5 Consistency and Data Protection
### Note:use the Kernek: "Python (wrangling)"

In [64]:
import pandas as pd
import os
import matplotlib.pyplot as plt

import hmac
import hashlib

### Read Data

In [3]:
file_path = os.path.join("..","data", "dirty-loan-data.csv")
df = pd.read_csv(file_path,low_memory = False)



## Integrity

### Data Types: Convert to date last_pymnt_d
HINT: to_datetime & format='%b-%Y' 

In [11]:
df.last_pymnt_d.head(3)

0    Jan-2015
1    Jan-2016
2    Sep-2015
Name: last_pymnt_d, dtype: object

0       2015-01-01
1       2016-01-01
2       2015-09-01
3       2014-01-01
4       2012-11-01
           ...    
17842   2013-06-01
17843   2014-05-01
17844   2013-04-01
17845   2014-11-01
17846   2014-11-01
Name: last_pymnt_d, Length: 17847, dtype: datetime64[ns]

### What to do here?

In [12]:
df.emp_length.head(5)

0     5 years
1    < 1 year
2     5 years
3     6 years
4     3 years
Name: emp_length, dtype: object

## Consistency

### Inconsistent categories

### Golden Reference / Authoritative Set
df['purpose'].unique().tolist()

### Suppose the Golden Reference is given

In [29]:
golden_reference_categories = ['debt_consolidation',
 'credit_card',
 'home_improvement',
 'other',
 'major_purchase',
 'car',
 'small_business',
 'medical',
 'moving',
 'vacation']

### Please Implement below function to identify values in df that are not in the golden_reference_categories


In [30]:
def find_inconsistent_categories(df,col,reference_categories):
    """ Find Inconsisten categories 
        Parameters:
        -----------
        df : pandas.DataFrame
        The input DataFrame to analyze 
        Returns:
        --------
        List of Columns name
    """
    

###       Execute it for "purpose": Which categories are not ok??

In [ ]:
find_inconsistent_categories(df,'purpose',golden_reference_categories)

## Pseudoanonymisation

### Simple obfuscation: Masking with RegExp

In [37]:
df['url'].str.replace(r'(?<=^.{4}).*', '*', regex=True).head(5)


0    http*
1    http*
2    http*
3    http*
4    http*
Name: url, dtype: object

### Examples of Python Packages

### Anonymizedf

In [40]:
from anonymizedf.anonymizedf import anonymize


In [56]:
an = anonymize(df[['id','loan_amnt','funded_amnt','purpose']].head(10))



In [62]:
an_df = (
    an
    .fake_ids("id", chaining=True)
    .fake_categories("purpose", chaining=True)
    .show_data_frame()
)
fake_df

,id,loan_amnt,funded_amnt,purpose,Fake_id,Fake_purpose
0,1069453,11000.0,11000.0,debt_consolidation,CEHF34821073307765,purpose 1
1,1067874,6000.0,6000.0,major_purchase,ZUGJ28999336352372,purpose 2
2,1057633,8450.0,8450.0,debt_consolidation,QBJW67050686087225,purpose 1
3,1061877,20000.0,20000.0,debt_consolidation,BQRE43672669800389,purpose 1
4,1056150,13200.0,8975.0,small_business,MQQA27892501726194,purpose 3
5,1065026,8000.0,8000.0,credit_card,LSAR71942003347551,purpose 4
6,1065199,3500.0,3500.0,debt_consolidation,KDGK49143861004070,purpose 1
7,1064567,12000.0,12000.0,debt_consolidation,CXVE33595273303499,purpose 1
8,1062032,4000.0,4000.0,credit_card,BZFH41872087171414,purpose 4
9,988402,18500.0,15775.0,debt_consolidation,LDFJ77818009338382,purpose 1


#### How would you use it in PROD?
- Delete original Columns?
- Save such info as a mapping CSV for the future?
- Is it reproducible?

### hmac,hashlib

#### First define a private key that will be separately and securely stored

In [70]:
private_key = os.urandom(24)

In [75]:
def anonymize(value, private_key):
    value_bytes = str(value).encode('utf-8')
    private_key_bytes = str(private_key).encode('utf-8')
    hash_object = hmac.new(private_key_bytes, value_bytes, hashlib.sha256)
    hashed_member_id = hash_object.hexdigest()
    return hashed_member_id

In [76]:
original_member_id = 1303701
anonymized_member_id = anonymize(original_member_id, private_key)

print(f"Original Member ID: {original_member_id}")
print(f"Anonymized Member ID: {anonymized_member_id}")

Original Member ID: 1303701
Anonymized Member ID: bb941373989a5af14d128a5d63bebec99f944166bdb17c7e62812a41ef30d166
